In [1]:
#!pip install transformers
import transformers
print(transformers.__version__)

4.25.1


In [1]:
#from collections.abc import Iterable

In [3]:
#QG
from pipelines import pipeline
#PDF
from PyPDF2 import PdfReader
#Document store
from haystack.document_stores import PineconeDocumentStore
from haystack import Document
#torch
import torch
from haystack.utils import print_answers
import pickle

## QA pipline

In [4]:
#import pinecone

#pinecone.init(api_key="dcb09f17-2a00-4e05-840f-eb7636bc7320", environment="us-east1-gcp")

In [4]:
from haystack.document_stores import PineconeDocumentStore

document_store = PineconeDocumentStore(
    api_key='d27dc937-4ba8-45a8-8418-15e47115a033',
    index='haystack-lfqa',
    similarity="cosine",
    embedding_dim=768
)

In [5]:
print(document_store.metric_type)
print(document_store.get_document_count())
print(document_store.get_embedding_count())

cosine
21504
21504


In [7]:
#Retriever
from haystack.nodes import EmbeddingRetriever

retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="flax-sentence-embeddings/all_datasets_v3_mpnet-base",
    model_format="sentence_transformers"
)

In [8]:
#Generator

from haystack.nodes import Seq2SeqGenerator

generator = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa")

In [9]:
#Our QA piline

from haystack.pipelines import GenerativeQAPipeline

pipe = GenerativeQAPipeline(generator, retriever)

In [30]:
type(pipe)

haystack.pipelines.standard_pipelines.GenerativeQAPipeline

### Extracting Text from PDF

In [10]:
def text_from_pdf(path: str) -> str:
    reader = PdfReader(path)
    pages = reader.pages
    doc = str()
    for page in pages:
        doc = doc + page.extract_text() + " "
    return doc

In [12]:
def text_from_pdf_qg(path: str, nlp):
    reader = PdfReader(path)
    pages = reader.pages
    questions = []
    for page in pages:
        doc = page.extract_text()
        quest = nlp(doc)
        for item in quest:
            questions.append(item)
    return questions

### Updating our document store

In [13]:
def update_doc_store(doc: str, article_title : str = "None", section_title : str = "None" ):
    docs = []
    doc = Document(
            content= doc,
            meta={
                "article_title": article_title,
                'section_title': section_title
            }
        )
    docs.append(doc)
    embeds = retriever.embed_documents(docs)
    doc.embedding = embeds[0]
    document_store.write_documents(docs)

## QG

In [14]:
nlp = pipeline("e2e-qg")

In [19]:
file_name = "01.pdf"
#doc = text_from_pdf(file_name)

In [20]:
questions = text_from_pdf_qg(file_name,nlp)

In [21]:
questions

['What is the name of the group that is trading as the Taylor & Francis Group?',
 'What is a name for the company that is based in China?',
 'How many AGB equations were developed after all necessary biomass calculations were done?',
 'What is the name of the group that is widely distributed in dry forest and forest margins?',
 'Which species has many economic and ecological functions, but its environmental uses are less assessed?',
 'The best models selected have higher R2-adj and lower residual standard error?',
 'What is an important species in evergreen montane scrub?',
 'What is edible and sometimes used to extract oil?',
 'Where are leaves, twigs, and wood used to fumigate pots for milk, TELLA, and TEJ?',
 'What are the equa- tions developed to estimate biomass components of trees and shrubs in various ecosystems in the world?',
 'What can the choice of the allometric model overestimate the greenhouse gas by 40% in the Congo Basin?',
 'The site- and species-specific equation has 

In [23]:
import re
match = " my email addres is anouar.taizoukt@um6p.ma anouar@gmail.com jj"
match = re.sub(r'[\w.+-]+@[\w-]+\.[\w.-]+',"" ,match)
match

' my email addres is   jj'

In [119]:
update_doc_store(doc)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Writing Documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [120]:
print(document_store.get_embedding_count())

21504


In [40]:
result = pipe.run(
    query="The moon's gravity causes what?",
    params={
        "Retriever": {"top_k": 3},
        "Generator": {"top_k": 1}
    }
)

print_answers(result, details="minimum")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: The moon's gravity causes what?
Answers:
[{'answer': "The Moon's gravity causes the tides on the surface of the Earth."}]


In [47]:
answer_dic = result["answers"][0].to_dict()
answer_dic["answer"]

"The Moon's gravity causes the tides on the surface of the Earth."

In [123]:
for doc in result['documents'][1:]:
    print(doc.content, end='\n---\n')

there is a conspicuous lack of both mathematical theory and controlled experiment, as we know it. These later became decisive factors in forming modern science, and their early application came to be known as classical mechanics.
In his Elementa super demonstrationem ponderum, medieval mathematician Jordanus de Nemore introduced the concept of "positional gravity" and the use of component forces.
The first published causal explanation of the motions of planets was Johannes Kepler's Astronomia nova, published in 1609. He concluded, based on Tycho Brahe's observations on the orbit of Mars, that the planet's orbits were ellipses. This break with ancient thought was happening
---
the rotation axis of the Earth; this much smaller effect is due to the uneven distribution of the Earth's mass. To make computations feasible, scientists defined ellipsoids of revolution; a given ellipsoid would be a good compromise for measurements in a given area, such as a country or continent. The difference b